In [4]:
import polars as pl

In [5]:
pitching = pl. read_csv("./Data/lahman_1871-2023_csv/Pitching.csv")
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [52]:
import pandas as pd
pitching = pd.read_csv("./Data/lahman_1871-2023_csv/Pitching.csv")
modern_pitching = pitching[pitching['yearID'] >= 1946]
grouped_era = modern_pitching.groupby(['yearID', 'lgID']).agg({
    'ER': 'sum',   
    'IPouts': 'sum'  
}).reset_index()
grouped_era['ERA'] = (grouped_era['ER'] / grouped_era['IPouts']) * 27
print(grouped_era.head())


   yearID lgID    ER  IPouts       ERA
0    1946   AL  4306   33186  3.503345
1    1946   NL  4202   33254  3.411740
2    1947   AL  4560   33228  3.705309
3    1947   NL  4956   32902  4.066987
4    1948   AL  5223   32929  4.282578


In [53]:
def dh_rule(year, league):
    if league == 'AL' and year >= 1973:
        return 'AL_with_DH'
    elif league == 'NL' and year >= 2022:
        return 'NL_with_DH'
    else:
        return 'No_DH'
grouped_era['DH_rule'] = grouped_era.apply(lambda row: dh_rule(row['yearID'], row['lgID']), axis=1)
print(grouped_era.head())


   yearID lgID    ER  IPouts       ERA DH_rule
0    1946   AL  4306   33186  3.503345   No_DH
1    1946   NL  4202   33254  3.411740   No_DH
2    1947   AL  4560   33228  3.705309   No_DH
3    1947   NL  4956   32902  4.066987   No_DH
4    1948   AL  5223   32929  4.282578   No_DH


In [56]:
batting = pd.read_csv("./Data/lahman_1871-2023_csv/Batting.csv")
batting['1B'] = batting['H'] - (batting['2B'] + batting['3B'] + batting['HR'])
grouped_slg = batting.groupby(['yearID', 'lgID']).agg({
    '1B': 'sum',
    '2B': 'sum',
    '3B': 'sum',
    'HR': 'sum',
    'AB': 'sum'  
}).reset_index()
grouped_slg['Total_Bases'] = (grouped_slg['1B'] + 
                              2 * grouped_slg['2B'] + 
                              3 * grouped_slg['3B'] + 
                              4 * grouped_slg['HR'])

grouped_slg['SLG'] = grouped_slg['Total_Bases'] / grouped_slg['AB']
grouped_slg['DH_rule'] = grouped_slg.apply(lambda row: dh_rule(row['yearID'], row['lgID']), axis=1)
print(grouped_slg.head())


   yearID lgID    1B   2B   3B  HR     AB  Total_Bases       SLG DH_rule
0    1876   NL  4484  633  181  40  20121         6453  0.320710   No_DH
1    1877   NL  3046  431  204  24  13667         4616  0.337748   No_DH
2    1878   NL  2903  481  132  23  13644         4353  0.319041   No_DH
3    1879   NL  4838  958  317  58  24155         7937  0.328586   No_DH
4    1880   NL  4576  980  328  62  24301         7768  0.319658   No_DH
